# Importing Required Libraries

In [1]:
import os
import torch
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.metrics import classification_report

# Mounting Drives

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


# Defining Constants

In [3]:
data_dir = "/content/drive/MyDrive/AMLAssignment5/csv_files"

train = pd.read_csv(os.path.join(data_dir, "train.csv"), encoding="ISO-8859-1")
test = pd.read_csv(os.path.join(data_dir, "test.csv"), encoding="ISO-8859-1")

train_texts = train['text']
train_labels = LabelEncoder().fit_transform(train['sentiment'])

test_texts = test['text']
test_labels = LabelEncoder().fit_transform(test['sentiment'])

# Defining Model Related Variables

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # negative, neutral and positive classes

train_encodings = tokenizer([str(x) for x in train_texts], truncation=True, padding=True)
test_encodings = tokenizer([str(x) for x in test_texts], truncation=True, padding=True)

optimizer = AdamW(model.parameters(), lr=5e-5)

train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels.tolist()),
)

test_dataset = TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels.tolist()),
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Setting device (GPU if available)

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

# Training Model

In [6]:
model = model.to(device)

for epoch in range(10):
    model.train()
    for batch in tqdm(train_loader, desc = f"Epoch: {epoch + 1}"):
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

Epoch: 1:   0%|          | 0/1133 [00:00<?, ?it/s]

Epoch: 2:   0%|          | 0/1133 [00:00<?, ?it/s]

Epoch: 3:   0%|          | 0/1133 [00:00<?, ?it/s]

Epoch: 4:   0%|          | 0/1133 [00:00<?, ?it/s]

Epoch: 5:   0%|          | 0/1133 [00:00<?, ?it/s]

Epoch: 6:   0%|          | 0/1133 [00:00<?, ?it/s]

Epoch: 7:   0%|          | 0/1133 [00:00<?, ?it/s]

Epoch: 8:   0%|          | 0/1133 [00:00<?, ?it/s]

Epoch: 9:   0%|          | 0/1133 [00:00<?, ?it/s]

Epoch: 10:   0%|          | 0/1133 [00:00<?, ?it/s]

# Evaluating Model

In [7]:
model.eval()

test_preds = []
test_labels = []

for batch in tqdm(test_loader, desc = "Testing Model"):
    input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, dim=1).detach().cpu().numpy()
    test_preds.extend(preds)
    test_labels.extend(labels.detach().cpu().numpy())

print(classification_report(test_labels, test_preds))

Testing Model:   0%|          | 0/602 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.76      0.78      0.77      1001
           1       0.39      0.76      0.52      1430
           2       0.85      0.80      0.82      1103
           3       0.00      0.00      0.00      1281

    accuracy                           0.57      4815
   macro avg       0.50      0.58      0.53      4815
weighted avg       0.47      0.57      0.50      4815



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
